In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import re
import datetime as dt
%matplotlib inline

In [2]:
'''
Importing the text file which contains the chat of the group
in read mode using utf-8 encoding!
This is my clg group chat,it contains chats from 2018,so its a very big
file,will preview it after we've done adding regular expressions!

'''

f = open('dummy_chat_file.txt','r',encoding='utf-8')

data = f.read()

In [3]:
dummy = data.split('\n')
dummy

['30/12/20, 11:57 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.',
 '30/12/20, 11:57 - ABC: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aenean posuere libero sem, eu commodo velit condimentum eget. Nam tortor elit, dictum ut od',
 '30/12/20, 11:59 - PQR: Cras luctus, sapien tempor elementum ullamcorper, lacus orci facilisis augue, non ultricies nibh nibh et magna. Nullam a sagittis orci',
 '29/01/21, 12:11 - PQR: Praesent finibus vel ex et consectetur. Suspendisse potenti. Suspendisse faucibus neque nec diam facilisis dapibus.',
 '29/01/21, 12:11 - PQR: Phasellus vehicula ipsum eu velit eleifend egestas.',
 '30/01/21, 9:46 - ABC: Morbi pharetra condimentum massa, in lobortis lectus consequat eget. Vivamus lacinia porttitor efficitur. Cras sit amet dignissim mi, nec fringilla massa.',
 '']

In [4]:
'''
Example for dry run : '16/08/18, 20:09 - '

\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s

the first part "d{1,2}" this means here we can have one or 2 digits
then after we have "/"
then after we have "d{1,2}" again,same we can have one or 2 digits
then after we have "/"
then after we have "d{2,4}" here we check for 2 or more digits
then we have a space represented as "\s"
then we have hour representation it can take either one or 2 digits 
then we have a seperator ":" 
and the pattern repeats so on..

'''

# username and message in different col

pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
messages = re.split(pattern,data)[1:]
print(len(messages))

## extracting dates as well in different col

dates = re.findall(pattern,data)
print(len(dates))

6
6


In [5]:
# this is a simple time string,we will appply this kind of transformation
# inorder to get the date and the time for our analysis

string = '16/08/18, 20:09 - '
string = string.split(',')
date,time = string[0],string[1]
time = time.split('-')
time = time[0].strip()
print(date+" and "+time)

16/08/18 and 20:09


In [6]:
# this function is to seperate the time and date

def gettimeanddate(string):
    string = string.split(',')
    date,time = string[0],string[1]
    time = time.split('-')
    time = time[0].strip()
    
    return date+" "+time




In [7]:
# creating a dataframe for messages and their dates

df = pd.DataFrame({'user_messages':messages,
                   'message_date':dates})

df['message_date'] = df['message_date'].apply(lambda text:gettimeanddate(text))
df.rename(columns={'message_date':'date'},inplace=True)

df.head()

,user_messages,date
0,Messages and calls are end-to-end encrypted. N...,30/12/20 11:57
1,"ABC: Lorem ipsum dolor sit amet, consectetur a...",30/12/20 11:57
2,"PQR: Cras luctus, sapien tempor elementum ulla...",30/12/20 11:59
3,PQR: Praesent finibus vel ex et consectetur. S...,29/01/21 12:11
4,PQR: Phasellus vehicula ipsum eu velit eleifen...,29/01/21 12:11


In [8]:
df['user_messages']


0    Messages and calls are end-to-end encrypted. N...
1    ABC: Lorem ipsum dolor sit amet, consectetur a...
2    PQR: Cras luctus, sapien tempor elementum ulla...
3    PQR: Praesent finibus vel ex et consectetur. S...
4    PQR: Phasellus vehicula ipsum eu velit eleifen...
5    ABC: Morbi pharetra condimentum massa, in lobo...
Name: user_messages, dtype: object

In [9]:
'''
if we observe the usermessage,we find that the 
username is attached with the user message,so i need to get rid
of this,for that i will use the concept of the regular expression

sample of the regular expression output

['', 'Chinmayee', 'Hello this is official grup\n']

so here we can see that the username is at index 1 and message is at index2

sometimes it happens that we get the group notification,so 
for that case we have to handle it accordingly!


 Chinmayee: Hello this is official grup
 
 Chinmayee added you\n-->['Chinmayee added you','\n']

'''

users = []
messages = []

for message in df['user_messages']:
    
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(entry[2])

        
    else:
        users.append('Group Notification')
        messages.append(entry[0])
        
        
df['User'] = users
df['message'] = messages


def getstring(text):
    return text.split('\n')[0]

df['message'] = df['message'].apply(lambda text:getstring(text))


df = df.drop(['user_messages'],axis=1)
df = df[['message','date','User']]

df = df.rename(columns={'message':'Message',
                       'date':'Date'})



df.head()

,Message,Date,User
0,Messages and calls are end-to-end encrypted. N...,30/12/20 11:57,Group Notification
1,"Lorem ipsum dolor sit amet, consectetur adipis...",30/12/20 11:57,ABC
2,"Cras luctus, sapien tempor elementum ullamcorp...",30/12/20 11:59,PQR
3,Praesent finibus vel ex et consectetur. Suspen...,29/01/21 12:11,PQR
4,Phasellus vehicula ipsum eu velit eleifend ege...,29/01/21 12:11,PQR


In [11]:
df.shape

(6, 3)

In [12]:
df['Only date'] = pd.to_datetime(df['Date']).dt.date

df['Year'] = pd.to_datetime(df['Date']).dt.year

df['Month_num'] = pd.to_datetime(df['Date']).dt.month

df['Month'] = pd.to_datetime(df['Date']).dt.month_name()

df['Day'] = pd.to_datetime(df['Date']).dt.day

df['Day_name'] = pd.to_datetime(df['Date']).dt.day_name()


df['Hour'] = pd.to_datetime(df['Date']).dt.hour

df['Minute'] = pd.to_datetime(df['Date']).dt.minute


df.head()

,Message,Date,User,Only date,Year,Month_num,Month,Day,Day_name,Hour,Minute
0,Messages and calls are end-to-end encrypted. N...,30/12/20 11:57,Group Notification,2020-12-30,2020,12,December,30,Wednesday,11,57
1,"Lorem ipsum dolor sit amet, consectetur adipis...",30/12/20 11:57,ABC,2020-12-30,2020,12,December,30,Wednesday,11,57
2,"Cras luctus, sapien tempor elementum ullamcorp...",30/12/20 11:59,PQR,2020-12-30,2020,12,December,30,Wednesday,11,59
3,Praesent finibus vel ex et consectetur. Suspen...,29/01/21 12:11,PQR,2021-01-29,2021,1,January,29,Friday,12,11
4,Phasellus vehicula ipsum eu velit eleifend ege...,29/01/21 12:11,PQR,2021-01-29,2021,1,January,29,Friday,12,11


In [13]:
df[df['User'] == 'ABC']

,Message,Date,User,Only date,Year,Month_num,Month,Day,Day_name,Hour,Minute
1,"Lorem ipsum dolor sit amet, consectetur adipis...",30/12/20 11:57,ABC,2020-12-30,2020,12,December,30,Wednesday,11,57
5,"Morbi pharetra condimentum massa, in lobortis ...",30/01/21 9:46,ABC,2021-01-30,2021,1,January,30,Saturday,9,46


In [14]:
words = []
for message in df['Message']:
    words.extend(message.split())
    
print(len(words))

111


In [15]:
words

['Messages',
 'and',
 'calls',
 'are',
 'end-to-end',
 'encrypted.',
 'No',
 'one',
 'outside',
 'of',
 'this',
 'chat,',
 'not',
 'even',
 'WhatsApp,',
 'can',
 'read',
 'or',
 'listen',
 'to',
 'them.',
 'Tap',
 'to',
 'learn',
 'more.',
 'Lorem',
 'ipsum',
 'dolor',
 'sit',
 'amet,',
 'consectetur',
 'adipiscing',
 'elit.',
 'Aenean',
 'posuere',
 'libero',
 'sem,',
 'eu',
 'commodo',
 'velit',
 'condimentum',
 'eget.',
 'Nam',
 'tortor',
 'elit,',
 'dictum',
 'ut',
 'od',
 'Cras',
 'luctus,',
 'sapien',
 'tempor',
 'elementum',
 'ullamcorper,',
 'lacus',
 'orci',
 'facilisis',
 'augue,',
 'non',
 'ultricies',
 'nibh',
 'nibh',
 'et',
 'magna.',
 'Nullam',
 'a',
 'sagittis',
 'orci',
 'Praesent',
 'finibus',
 'vel',
 'ex',
 'et',
 'consectetur.',
 'Suspendisse',
 'potenti.',
 'Suspendisse',
 'faucibus',
 'neque',
 'nec',
 'diam',
 'facilisis',
 'dapibus.',
 'Phasellus',
 'vehicula',
 'ipsum',
 'eu',
 'velit',
 'eleifend',
 'egestas.',
 'Morbi',
 'pharetra',
 'condimentum',
 'massa,'

In [16]:
df[(df['Message'] == '<Media omitted>')&(df['User'] == 'Jagss')].shape

(0, 11)